In [ ]:
import pandas as pd
from cdcqr.common.config import LOCAL_DATA_DIR
from cdcqr.common.utils import load_df, save, resample_pv
from cdcqr.common.stats_utils import remove_outlier
import sys
sys.path.append('/github/cryptoderiv-quant/')
import matplotlib.pyplot as plt

In [ ]:
from ct.alfafactory import runsignallight
from ct.fs import addma

In [ ]:
a1 = 'BTCUSD_PERP@binance'

In [ ]:
df_btc = load_df(a1)
df_btc.head()

In [ ]:
res=[]
for freq in ['1Min', '3Min', '5Min', '10Min', '30Min']:
    price = df_btc['c'].resample(freq).last().to_frame()
    num_period = len(price)
    for lbw in [5,10,20,30,60,120]:
        price['ma'] = price['c'].rolling(lbw).mean()
        for nstd in [2,3,4]:
            price['std'] = price['c'].rolling(lbw).std()
            price['hi'] = price['ma']+nstd*price['std']
            price['lo'] = price['ma']-nstd*price['std']
            for short in [True, False]:
                if short:
                    signal = price['c']>price['hi']
                else:
                    signal = price['c']<price['lo']
                num_signal = signal.sum()
                
                for n in [100,200,500,1000]:
                    ressig=runsignallight(price,signal=signal,n=n,short=short,fees=0.0007)
                    resd={"freq":freq, "lbw":lbw, 'short':short,'nstd':nstd,'n':n,"p":ressig, 'num_signal':num_signal, 'num_period':num_period}        
                    res.append(resd)

In [ ]:
pd.DataFrame(res).query('num_signal>50').groupby(['nstd','freq','lbw'])['p'].mean()

In [ ]:
pd.DataFrame(res).query('num_signal>50').pv()